In [ ]:
from get_granule_image import get_granule_image

scene_name = 'Karangetang_0_08'
granule_idx = 1

image = get_granule_image(desired_scene_name=scene_name,desired_granule_idx=granule_idx)

In [ ]:
from coregistration_superglue_multiband import SuperGlue_registration
import os

from constants import thraws_data_path

data_path = thraws_data_path

desired_scene_name = "Kenya_0"
desired_granule_idx = 2

bands_list = ["B12","B11","B8A"]
scene_groups_paths = [
        os.path.join(data_path,d) for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))
    ]

from pyraws.raw.raw_event import Raw_event


# scene_name_condition = False
# Process creating loop
for i,scene_group_path in enumerate(scene_groups_paths):
    
        scenes_paths = [
                os.path.join(scene_group_path,scene_name) for scene_name in os.listdir(scene_group_path) if os.path.isdir(os.path.join(scene_group_path, scene_name))
            ]
        
        for scene_path in scenes_paths:
            granules_paths = [
                os.path.join(scene_path,granule_name) for granule_name in os.listdir(scene_path) if os.path.isdir(os.path.join(scene_path, granule_name))
            ]
            scene_name = os.path.basename(scene_path)

            if scene_name == desired_scene_name:
                # print(scene_name)
                for granule_idx in range(len(granules_paths)):

                    if granule_idx == desired_granule_idx:
                        raw_coreg_granule,_,coarse_status = SuperGlue_registration(bands_list=bands_list,event_path=scene_path,granule_idx=granule_idx,verbose=True)

                        raw_event = Raw_event()

                        # Read event from data
                        raw_event.from_path(  # Path to the event
                            raw_dir_path=scene_path,
                            # Bands to open. Leave to None to use all the bands.
                            bands_list=bands_list,
                            # If True, verbose mode is on.
                            verbose=False,
                        )

                        raw_granule = raw_event.get_granule(granule_idx).as_tensor()



This granule: 2
Stackable granules: [[0, 2], [1, 3], [2, 4], [3, 5]]
Granule before: 0
Granule after: 4
Bands shifts:  [[-198.0, 0.0], [-15.0, -12.0]]


In [ ]:
import sys
sys.path.insert(1,'..')
from utils import normalize_to_0_to_1
import numpy as np
granule = normalize_to_0_to_1(raw_granule).numpy()
granule_f16 = granule.astype(np.float16)


def apply_shift(band, shifts):
    band_shifted = np.zeros_like(band)
    shifts = [int(shifts[0]), int(shifts[1])]
    if (shifts[0] == 0) and (shifts[1] == 0):
        band_shifted = band
    elif (shifts[0] == 0) and (shifts[1] < 0):
        band_shifted[:, : int(shifts[1])] = band[:, -int(shifts[1]) :]
    elif (shifts[0] == 0) and (shifts[1] > 0):
        band_shifted[:, int(shifts[1]) :] = band[:, : -int(shifts[1])]
    elif (shifts[0] < 0) and (shifts[1] == 0):
        band_shifted[: int(shifts[0]), :] = band[-int(shifts[0]) :, :]
    elif (shifts[0] > 0) and (shifts[1] == 0):
        band_shifted[int(shifts[0]) :, :] = band[: -int(shifts[0]), :]
    elif (shifts[0] > 0) and (shifts[1] > 0):
        band_shifted[int(shifts[0]) :, int(shifts[1]) :] = band[
            : -int(shifts[0]), : -int(shifts[1])
        ]
    elif (shifts[0] > 0) and (shifts[1] < 0):
        band_shifted[int(shifts[0]) :, : int(shifts[1])] = band[
            : -int(shifts[0]), -int(shifts[1]) :
        ]
    elif (shifts[0] < 0) and (shifts[1] > 0):
        band_shifted[: int(shifts[0]), int(shifts[1]) :] = band[
            -int(shifts[0]) :, : -int(shifts[1])
        ]
    else:
        band_shifted[: int(shifts[0]), : int(shifts[1])] = band[
            -int(shifts[0]) :, -int(shifts[1]) :
        ]

    return band_shifted


shift_list = [[199.0, -1.0], [25.0, -3.0]]


aligned_b11 = apply_shift(granule[:,:,1],shifts=shift_list[0])
aligned_b8a = apply_shift(granule[:,::-1,2],shifts=shift_list[1])

granule_coregistered = np.dstack((granule[:,:,0],aligned_b11,aligned_b8a))




In [ ]:
from utils import save_as_binary

save_as_binary(output_path='NIR_SWIR_original_Kenya_0_G2_testing.bin',image=granule)

In [ ]:
import cv2 

cv2.imshow('Image B12',granule[:,:,0])
cv2.imshow('Image B11',granule[:,:,1])
cv2.imshow('Image B8A',granule[:,:,2])
cv2.imshow('Co-registered',granule_coregistered)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from constants import DATASET_PATH
import os
import pickle
rgb_path = os.path.join(DATASET_PATH,'images','event','RGB')
images = [os.path.join(rgb_path,rgb_image_name) for rgb_image_name in os.listdir(rgb_path)]
upper_thresholds = []
lower_thresholds = []
mean_thresholds = []
std_thresholds = []
for image_path in images:
    with open(image_path,'rb') as file:
        image = pickle.load(file)
    upper_thresholds.append(image.max())
    lower_thresholds.append(image.min())
    mean_thresholds.append(image.mean())
    std_thresholds.append(image.std())
print(max(upper_thresholds))
print(min(lower_thresholds))

0.8764347
0.0


In [ ]:
np.mean(mean_thresholds),np.std(std_thresholds)

(0.0873259, 0.035534315)

In [ ]:
import numpy as np

def normalize_0_1_float(band,band_max,band_min):
    # Function that converts 16bits images into 8bits images
    # band_max, band_min = band.max(), band.min()
    new_image = (((band - band_min) / ((band_max - band_min))) ).astype(np.float32)
    new_image[new_image >1] = 1
    return (((band - band_min) / ((band_max - band_min))) ).astype(np.float32)

def normalize(band):
    # Function that converts 16bits images into 8bits images
    band_max, band_min = band.max(), band.min()
    return (((band - band_min) / ((band_max - band_min))) * 255).astype(np.uint8)


def normalizeStd(image):
    """Normalize the input around the mean value and twice the standard deviation

    Attributes
    ----------
    image : array
        Image to be normalized with its mean and standard deviation.

    Output :
    ------
    Float32 image.

    Notes
    -----
    Applied normally in visualization purposes. The normalization is better than the normal min and max.
    """
    return (
        (image - (np.nanmean(image) - np.nanstd(image) * 2))
        / (
            (np.nanmean(image) + np.nanstd(image) * 2)
            - (np.nanmean(image) - np.nanstd(image) * 2)
        )
    ).astype(np.float32)

def normalizeStd_global(image,mean,std):
    """Normalize the input around the mean value and twice the standard deviation

    Attributes
    ----------
    image : array
        Image to be normalized with its mean and standard deviation.

    Output :
    ------
    Float32 image.

    Notes
    -----
    Applied normally in visualization purposes. The normalization is better than the normal min and max.
    """
    return (
        (image - (mean - std * 2))
        / (
            (mean + std * 2)
            - (mean - std * 2)
        )
    ).astype(np.float32)

upper_norm = max(upper_thresholds)
lower_norm = min(lower_thresholds)

import cv2

for image_path in images:
    with open(image_path,'rb') as file:
        image = pickle.load(file)
    norm_image = normalize_0_1_float(image,np.mean(upper_thresholds),lower_norm)
    norm_image_2 = normalize(image)
    norm_image_3 = normalizeStd(image)
    norm_image_4 = normalizeStd_global(image,np.mean(mean_thresholds),np.std(std_thresholds))
    # print(norm_image_4.max(),norm_image_4.min())
    cv2.imshow('Image',image)
    cv2.imshow('Normalized image',norm_image)
    cv2.imshow('Normalized image 2',norm_image_2)
    cv2.imshow('Normalized image 3',norm_image_3)
    cv2.imshow('Normalized image 4',norm_image_4)
    key = cv2.waitKey(0)

    if key == 27:
        cv2.destroyAllWindows()
        break
    cv2.destroyAllWindows()
    # if i >2:
    #     break

    

In [ ]:
import cv2 

print(image.shape)
cv2.imshow('Image',image[:,:,2])
cv2.waitKey(0)
cv2.destroyAllWindows()

(1152, 1296, 3)


In [ ]:
images_path = "/media/cristopher/SansDisk1TB/datasets/train_geo_split_weakly_B04_B03_B02_dataset_norm/test/images"

def read_binary_image(image_path: str,
                      dtype: type = None,
                      shape: list = None
                      ):
    with open(image_path, 'rb') as f:
        image_bin = f.read()
        if dtype:
            image = np.frombuffer(image_bin,dtype=dtype)
        else:
            image = np.frombuffer(image_bin)

        if shape:
            image = image.reshape((shape))

    return image

for image_name in os.listdir(images_path):
    image_path = os.path.join(images_path,image_name)
    image = read_binary_image(image_path=image_path,dtype=np.float32,shape=[256,256,3])
    
    cv2.imshow('Image',image)
    key = cv2.waitKey(0)

    if key == 27:
        cv2.destroyAllWindows()
        break
    cv2.destroyAllWindows()

In [ ]:
scene_name = 'Karangetang_0_10'
granule_idx = 4

image_2 = get_granule_image(desired_scene_name=scene_name,desired_granule_idx=granule_idx)


In [ ]:
import cv2 

# print(image_2.shape)
cv2.imshow('Image',image_2[:,:,1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
scene_name = 'Nyamulagira_04'
granule_idx = 5

image_3 = get_granule_image(desired_scene_name=scene_name,desired_granule_idx=granule_idx)


In [ ]:
import cv2 

# print(image_3.shape)
cv2.imshow('Image',image_3[:,:,1])
cv2.waitKey(0)
cv2.destroyAllWindows()